<a href="https://colab.research.google.com/github/anton-akulenko/HW02-DS/blob/main/DS_HW03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [388]:
import pandas as pd
import numpy as np
import requests
import seaborn as sns
import matplotlib.pyplot as plt
from io import StringIO
drive_link = "https://drive.google.com/file/d/1-rAa4XT4_fI0dOBlMNuE6a7jB0wln_Qo/view?usp=share_link"

file_id = drive_link.split("/")[-2]
export_link = f"https://drive.google.com/uc?id={file_id}&export=download"

response = requests.get(export_link)
response.encoding = 'UTF-8'
csv_data = StringIO(response.text)

data = pd.read_csv(csv_data)
data



,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


Prepare normalised data

In [389]:
def normalization(value):

  mean = np.mean(value)
  value_range = np.max(value) - np.min(value)
  result = []
  for x in value:
    norm_x = (x - mean)/value_range
    result.append(norm_x)

  return result

In [390]:
norm_df = pd.DataFrame()
norm_df['price'] = normalization(data.price)
norm_df['area'] = normalization(data.area)
norm_df['bedrooms'] = normalization(data.bedrooms)
norm_df['bathrooms'] = normalization(data.bathrooms)
norm_df

,price,area,bedrooms,bathrooms
0,0.738811,0.155977,0.206972,0.237920
1,0.647902,0.261818,0.206972,0.904587
2,0.647902,0.330547,0.006972,0.237920
3,0.644872,0.161475,0.206972,0.237920
4,0.575175,0.155977,0.206972,-0.095413
...,...,...,...,...
540,-0.255128,-0.147804,-0.193028,-0.095413
541,-0.259704,-0.189041,0.006972,-0.095413
542,-0.261189,-0.105192,-0.193028,-0.095413
543,-0.261189,-0.153989,0.006972,-0.095413


In [391]:
X = norm_df[['area', 'bedrooms', 'bathrooms']]
y = norm_df['price']
# Додавання стовпця з одиницями для константного додатку
X.insert(0, 'ones', 1)

In [392]:

X = np.array(X)
y = np.array(y)
w = np.array([0., 0., 0., 0.])


напишіть функцію гіпотези лінійної регресії у векторному вигляді;
створіть функцію для обчислення функції втрат у векторному вигляді;


In [393]:
def h(w, X):
  return X@w

In [394]:
def loss_function(y_true, y_pred):
    mse = np.mean((y_true - y_pred)**2)
    return mse

GD

In [395]:
def grad_descent_step (weights, grads, learning_rate= 0.001):
  w_0, w_1 = weights
  grad_0, grad_1 = grads

  w_0 = w_0 - learning_rate * grad_0
  w_1 = w_1 - learning_rate * grad_1

  return w_0, w_1

In [396]:
def grad_descent_step (weights, y):
  m = len(y)
  gradient = (1/m) * X.T.dot(X.dot(w) - y)

  return gradient

In [397]:
def grad_desc (w, df, num_iterations= 1000, alpha = 0.1, epsilon = 0.0000001):

    loss_history = []
    m = len(y)
    w_inter = []
    i = 0
    for i in range(num_iterations):
        gradient = (1/m) * X.T.dot(X.dot(w) - y)
        w = w - alpha * gradient
        w_inter.append(w)


        loss = loss_function(y, X.dot(w))
        loss_history.append(loss)

        if len(loss_history) > 1 and abs(loss_history[-1] - loss_history[-2]) < epsilon:
            break
    print(f"Iterations #: {i} Loss: {loss_history[-3:]}")
    return w_inter

In [398]:
w_analytical = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)


In [402]:
w_GD = grad_desc(w, norm_df, 10000, 0.1, epsilon = 0.00000001)
print("Параметри w з градієнтного спуску:", w_GD[-1])

print("Параметри w з аналітичного рішення:", w_analytical)

Iterations #: 1683 Loss: [0.01342938112230641, 0.013429371083084425, 0.013429361083844085]
Параметри w з градієнтного спуску: [4.72608700e-17 4.67108436e-01 1.81735951e-01 3.59496069e-01]
Параметри w з аналітичного рішення: [1.81175091e-17 4.77142690e-01 1.76112569e-01 3.60012857e-01]
